# Condensates for Trappist 1-e models from Bower et al. (2024)

In [1]:
import logging

import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from pathlib import Path
import sys

from atmodeller import debug_logger
from atmodeller.constraints import ElementMassConstraint, SystemConstraints, ActivityConstraint
from atmodeller.core import GasSpecies, Species, LiquidSpecies, SolidSpecies
from atmodeller.interior_atmosphere import Planet, InteriorAtmosphereSystem
from atmodeller.initial_solution import InitialSolutionSwitchRegressor, InitialSolutionLast
from atmodeller.output import Output
from atmodeller.solver import SolverScipy
from atmodeller.solution import ELEMENT_PREFIX

logger = debug_logger()
logger.setLevel(logging.INFO)

Package initialized with double precision (float64)


Parameters for the simulations

In [2]:
# Some thermo data not available at 280 K
equilibrium_temperature = 300 
# training_steps = 200

TRAPPIST-1e planet properties

Mass and radius measurements from Agol et al. (2021). Mantle mass is determined assuming the same mass proportion as Earth.

In [3]:
planet_mass = 4.133E24
trappist1e = Planet(surface_temperature=equilibrium_temperature, planet_mass=planet_mass, 
                    surface_radius=5.861E6, mantle_melt_fraction=0)

Species to consider, including condensed C and H2O

In [4]:
# Gas species in the high temperature atmosphere
H2O_g = GasSpecies("H2O")
H2_g = GasSpecies("H2")
O2_g = GasSpecies("O2")
CO_g = GasSpecies("CO")
CO2_g = GasSpecies("CO2")
CH4_g = GasSpecies("CH4")
N2_g = GasSpecies("N2")
NH3_g = GasSpecies("NH3")
S2_g = GasSpecies("S2")
H2S_g = GasSpecies("H2S")
SO2_g = GasSpecies("SO2")
SO_g = GasSpecies("SO")
Cl2_g = GasSpecies("Cl2")

# Also consider these condensates, which may or may not be stable, but the code will decide.
H2O_l = LiquidSpecies("H2O")
C_cr = SolidSpecies("C")
S_l = LiquidSpecies("S")
S_cr = SolidSpecies("S", thermodata_filename="S-002")

species = Species([H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g, S2_g, SO2_g, SO_g, Cl2_g, 
                   H2S_g, H2O_l, C_cr, S_l, S_cr])

[20:56:12 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies for H2O using thermodynamic data in JANAF
[20:56:12 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies for H2 using thermodynamic data in JANAF
[20:56:12 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies for O2 using thermodynamic data in JANAF
[20:56:12 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies for CO using thermodynamic data in JANAF
[20:56:12 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies for CO2 using thermodynamic data in JANAF
[20:56:12 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies for CH4 using thermodynamic data in JANAF
[20:56:12 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies for N2 using thermodynamic data in JANAF
[20:56:12 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies for H3N using thermodynamic data in JANAF
/Users/dan/Documents/academic/explanetology/

The high temperature atmosphere simulations provide the constraints for the condensed atmosphere simulations.

In [5]:
elements = ["C", "H", "O", "N", "S", "Cl"]

high_temp_surface_temperature = 1800
high_temp_mantle_melt_fraction = 10
high_temp_number_of_realisations = 500

trappist1e_high_temp_full_path = Path(f"t1e_{high_temp_surface_temperature}K_{high_temp_mantle_melt_fraction*100:0.0f}melt_{high_temp_number_of_realisations}its")
high_temp_output = Output.read_pickle(trappist1e_high_temp_full_path.with_suffix('.pkl'))
high_temp_data = high_temp_output(to_dataframes=True)

# Consolidate the mass constraints of the elements in the atmosphere into a dataframe
atmosphere_mass = 'atmosphere_mass'
mass_constraints = {}
for element in elements:
    mass_constraints[element] = high_temp_data[f"{ELEMENT_PREFIX}{element}"][atmosphere_mass]
mass_constraints_df = pd.DataFrame(mass_constraints)

# The approach here is to order the data so we can exploit the fact that the previous solution is
# (probably) a reasonable initial guess for the next solution
# Compute the pairwise distance matrix
distance_matrix = cdist(mass_constraints_df.values, mass_constraints_df.values, metric='euclidean')

# Compute the average distance for each row
average_distances = distance_matrix.mean(axis=1)

# Get the sorted order of the rows based on average distances
sorted_indices = np.argsort(average_distances)
sorted_indices = pd.Index(sorted_indices)

# Reorder the array based on the sorted indices
mass_constraints_df = mass_constraints_df.iloc[sorted_indices]

# Assemble the system constraints for each entry in the dataframe
constraints: list[SystemConstraints] = []
for nn, row in enumerate(mass_constraints_df.itertuples(index=True)):
    constraints_list: list = []
    for element in elements:
        constraints_list.append(ElementMassConstraint(element, getattr(row, element)))
    constraints_list.append(ActivityConstraint(H2O_l, 1.0))
    constraints_list.append(ActivityConstraint(C_cr, 1.0))
    constraints_list.append(ActivityConstraint(S_l, 1.0))
    constraints_list.append(ActivityConstraint(S_cr, 1.0))
    constraints.append(SystemConstraints(constraints_list))

[20:56:24 - atmodeller.output              - INFO     ] - Output: Reading data from t1e_1800K_1000melt_500its.pkl


Start with the default dictionary of initial guesses and then switch to a regressor to provide a training dataset.

In [ ]:
initial_solution = InitialSolutionLast(species=species, planet=trappist1e)
initial_solution_training = InitialSolutionSwitchRegressor(initial_solution, species=species, planet=trappist1e, 
    fit=True, switch_iteration=50, fit_batch_size=50, partial_fit=True, 
    partial_fit_batch_size=50)

In [ ]:
trappist1e_full_path = Path(
    f"t1e_{equilibrium_temperature}K_was{high_temp_mantle_melt_fraction*100:0.0f}melt_{high_temp_number_of_realisations}its")

In [ ]:
trappist1e_full = InteriorAtmosphereSystem(species=species, planet=trappist1e)
solver = SolverScipy(jac=True)

for nn, row in enumerate(mass_constraints_df.itertuples(index=True)):
    # Need the index to correlate the condensed atmospheres with their high temperature origin.
    index = row.Index
    extra = {'index': index}
    constraints2 = constraints[nn]
    trappist1e_full.solve(constraints2, initial_solution=initial_solution_training,
        extra_output=extra)

In [ ]:
trappist1e_full.output(file_prefix=trappist1e_full_path, to_pickle=True, to_excel=True)

In [ ]:
trappist1e_full.number_of_failed_solves

In [ ]:
sys.exit(0)

# Post-processing

Finally, we can correlate the high temperature atmospheres to the condensed atmospheres. We only include combinations which we know solved to within the tolerance.

In [ ]:
output_high_temperature = Output.read_pickle(f"{trappist1e_with_sols_path}.pkl")
output_cooled_path = "trappist1e_280K_5000its"
output_condensed = Output.read_pickle(f"{output_cooled_path}.pkl")

output_reordered = output_high_temperature.reorder(output_condensed, "extra", "index")

In [ ]:
output_reordered(to_pickle=True, to_excel=True)

Find models that could not solve.

In [ ]:
output_reordered = output_high_temperature.filter_by_index_notin(output_condensed, "extra", "index")

In [ ]:
output_reordered(to_pickle=True, to_excel=True)